# Language Model Inferences

Libraries

In [1]:
import os
import json
import random
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer, pipeline
from transformers import BertTokenizerFast, BertForSequenceClassification

2024-10-22 09:00:55.501328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_model_path = os.path.join("../../ai/Banking/bert-model")
sql_model_path = os.path.join("../../ai/SQL/model-t5")
support_model_path = os.path.join("../../ai/Support/model-t5")

In [3]:
base_model = BertForSequenceClassification.from_pretrained(
    base_model_path
)

In [4]:
base_tokenizer = BertTokenizerFast.from_pretrained(
    base_model_path
)

In [5]:
base_pipe = pipeline(
    task='sentiment-analysis',
    model=base_model, 
    tokenizer=base_tokenizer, 
    device='mps'
)

In [6]:
base_pipe("Hello")

[{'label': 'greeting', 'score': 0.9989669322967529}]

In [7]:
def predict(text):
    inputs = base_tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("mps")
    outputs = base_model(**inputs)
    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = base_model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label


In [8]:
text = "Hello"
predict(text)

(tensor([[9.9897e-01, 4.0382e-05, 5.0594e-05, 2.0533e-05, 4.7807e-05, 1.3049e-05,
          1.7133e-05, 2.4188e-05, 1.1959e-05, 2.3613e-05, 3.7294e-05, 3.9787e-05,
          4.8652e-05, 4.1899e-05, 1.0683e-05, 3.7709e-05, 1.7957e-05, 2.8868e-05,
          3.0518e-05, 2.7206e-05, 1.1888e-05, 3.5707e-05, 2.1373e-05, 1.7299e-05,
          1.1532e-05, 1.4921e-05, 1.5354e-05, 5.2564e-05, 9.3531e-06, 1.2270e-05,
          2.2896e-05, 4.4233e-05, 4.9501e-05, 2.9215e-05, 4.3846e-05, 3.2940e-05,
          2.5992e-05, 1.2399e-05]], device='mps:0', grad_fn=<SoftmaxBackward0>),
 tensor(0, device='mps:0'),
 'greeting')

Prepare data for more inference

In [9]:
def load_json_file(filename):
    with open(filename) as f:
        file=json.load(f)
    return file   

In [10]:
filename="intents.json"
intents=load_json_file(filename)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'whatsup', 'Hi there, can you help me with something?', 'Hello! I have a question. Can you assist me?', 'Hey, I need your help with something.', 'Good morning/afternoon/evening! Can you answer a question for me?', 'Greetings! I need your assistance with something.', 'Hi,can you provide me with some information?', 'Hello, can you help me with a task?', "Hey,I'm looking for some guidance. Can you help?", 'Good day! Can you assist me with a query?', 'Hi,I would like to ask you a question.', 'Hello', 'Greetings', 'Good morning', 'Good afternoon', 'Good evening', 'Salutations', 'Pleasure to chat with you', "It's great to talk with you", 'Hi chatbot', 'Hello chatbot', 'Hey chatbot', "What's new", "How's it going", "What's happening", 'What can you do for me', 'Can you help me', 'I need your help', 'I have a question', "I'm here to chat", "Let's talk"], 'responses': ['Hello!', 'Hi there, 

In [27]:
def base_response(text):
    response = ''
    score = base_pipe(text)[0]['score']
    tag = base_pipe(text)[0]['label']
    if score < 0.8:
        response += 'Dont know'
    else:
        label = base_model.config.label2id[base_pipe(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])
    return response, tag

In [28]:
base_response('Hi')

("I'm here to make your life easier, how can I assist you?", 'greeting')

In [11]:
def chat(pipe):
    
    print("CHATBOT: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..")
    print("Type 'quit' to exit the chat\n\n")
    
    text = input("User: ").strip().lower()
    
    while(text != 'quit'):

        score = pipe(text)[0]['score']
        
        if score < 0.8:
            print("Chatbot: Sorry I can't answer that\n\n")
            text = input("User: ").strip().lower()
            continue
        
        label = collator_model.config.label2id[pipe(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])
        
        print(f"CHATBOT: {response}\n\n")
            
        text = input("YOU: ").strip().lower()


In [12]:
chat(base_pipe)

CHATBOT: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..
Type 'quit' to exit the chat


CHATBOT: How may I be of assistance today?


CHATBOT: List attendance reports at location this month


CHATBOT: My pleasure




In [28]:
sql_model = AutoModelForSeq2SeqLM.from_pretrained(
    sql_model_path
)

In [29]:
sql_tokenizer = AutoTokenizer.from_pretrained(
    sql_model_path
)

In [40]:
def sql_response(query):
    prompt = "translate English to SQL: %s " % query
    features = sql_tokenizer([prompt], return_tensors='pt')
    output = sql_model.generate(
        input_ids=features['input_ids'],
        max_new_tokens=200
        )
    return sql_tokenizer.decode(output[0], skip_special_tokens=True)

In [41]:
query = "How many meetings do we have?"
sql_response(query)

'SELECT count(id) FROM meetings'

In [37]:
support_model = AutoModelForSeq2SeqLM.from_pretrained(support_model_path)

In [38]:
support_tokenizer = AutoTokenizer.from_pretrained(
    support_model_path,
)

In [34]:
support_pipe = pipeline(
    task='text-generation',
    model=support_model, 
    tokenizer=support_tokenizer, 
    device='mps'
)

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralFor

In [42]:
def support_response(question):
    prompt = "You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional. %s " % question
    features = support_tokenizer([prompt], return_tensors='pt')
    output = support_model.generate(
        input_ids=features['input_ids'],
        max_new_tokens=200
        )
    return support_tokenizer.decode(output[0], skip_special_tokens=True)

In [43]:
question = "i have a question about cancelling order {{Order Number}}"
support_response(question)

"I've realized you have a question regarding canceling order Order Number. I apologize for any inconvenience this may have caused you. I'm here to assist you with this. Please Complaint Support, and I'll provide you with a direct link. You can contact our customer support team at Customer Support Phone Number or through the Live Chat on our website at Website URL. We are committed to resolving this issue and ensuring your satisfaction. Thank you for bringing this to our attention, and I apologize for any inconvenience caused. Thank you for bringing this to our attention, and I'm unable to assist you with the cancellation process. Thank you for bringing this to our attention, and I apologize for any inconvenience caused. Thank you for bringing this to our attention, and I'm here to assist you. Thank you for bringing this matter to our attention,"